In [6]:
from typing import List
import pandas as pd
import config as cfg
import numpy as np
import pandas as pd
import math
import torch
import config as cfg
from sksurv.util import Surv
from sklearn.model_selection import train_test_split
from utility.survival import Survival
from tools.regressors import CoxPH, RSF, DeepSurv, DSM, BNNmcd
from tools.feature_selectors import PHSelector
from utility.builder import Builder
from tools.file_reader import FileReader
from tools.data_ETL import DataETL
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from lifelines.statistics import proportional_hazard_test
from tools.evaluator import LifelinesEvaluator
from tools.Evaluations.util import predict_median_survival_time
import config as cfg
from sklearn.preprocessing import StandardScaler
from tools.formatter import Formatter
from xgbse.non_parametric import calculate_kaplan_vectorized
from utility.survival import make_event_times


# Paths
DATASET_PATH_XJTU = "../data/XJTU-SY/csv/"
RAW_DATA_PATH_XJTU = ["../data/XJTU-SY/35Hz12kN/", "../data/XJTU-SY/37.5Hz11kN/", "../data/XJTU-SY/40Hz10kN/"]
RESULTS_PATH = "../results"
PLOTS_PATH = "../plots"
RESULT_PATH_XJTU= "../data/XJTU-SY/results/"

NEW_DATASET = False
DATASET = "xjtu"
TYPE = "bootstrap"  # not_correlated
CONDITION_SELECTOR = 0 # 0, 1, 2
PERCENTAGE = 0.1
TEST_SIZE = 0.3
N_POST_SAMPLES = 1000 # number of samples to draw from MCD posterior
N_CONDITION = len(RAW_DATA_PATH_XJTU)
PLOTS_PATH = PLOTS_PATH

if TYPE == "bootstrap":
    N_BOOT = 0
    DATA_TYPE = "bootstrap"
else:
    N_BOOT = 0
    DATA_TYPE = "not_bootstrap"

N_CONDITION = len(RAW_DATA_PATH_XJTU)
N_BEARING = cfg.N_REAL_BEARING_XJTU
N_SPLITS = 5
TRAIN_SIZE = 1
CENSORING = cfg.CENSORING_LEVEL

#For the first time running, a NEW_DATASET is needed
if NEW_DATASET== True:
    Builder(DATASET, N_BOOT, TYPE).build_new_dataset(bootstrap=N_BOOT)

survival = Survival()
data_util = DataETL(DATASET, N_BOOT)

#Extract information from the dataset selected from the config file
cov_group = []
boot_group = []
info_group = []
for test_cond in range (0, N_CONDITION):
    cov, boot, info_pack = FileReader(DATASET, DATASET_PATH_XJTU, TYPE).read_data(test_cond, N_BOOT)
    cov_group.append(cov)
    boot_group.append(boot)
    info_group.append(info_pack)

In [71]:
data = cov_group[0].iloc[:, :25].dropna()
data['B19_Survival_time'] = range(119, 0, -1)
data['Time'] = range(0, 119, 1)
data['Id'] = 19

In [72]:
def series_to_moving_average(df_ts: pd.DataFrame,
                             window_len: int,
                             lag: int,
                             lbl_cols: List) -> pd.DataFrame:
    cols = list(df_ts.columns.drop(lbl_cols))
    total_df = pd.DataFrame()
    for ft_col in cols:
        roll = df_ts.groupby(['Id'])[ft_col].rolling(window_len)
        ma = roll.mean().shift(lag).reset_index(0, drop=True)
        total_df[ft_col] = ma
    #total_df = total_df.dropna().reset_index(drop=True)
    return total_df

In [73]:
data

,B19_mean,B19_std,B19_skew,B19_kurtosis,B19_entropy,B19_rms,B19_max,B19_p2p,B19_crest,B19_clearence,...,B19_Fo,B19_Fi,B19_Fr,B19_Frp,B19_Fca,B19_noise,B19_Event,B19_Survival_time,Time,Id
0,0.479684,0.604529,0.036587,0.394161,5.084863,0.604645,4.136920,7.663488,6.841896,0.405677,...,0.000048,0.000042,0.000055,0.000078,0.000088,0.000057,False,119,0,19
1,0.496474,0.627605,0.024118,0.550424,5.004417,0.627667,4.627383,8.613884,7.372348,0.419879,...,0.000091,0.000075,0.000055,0.000061,0.000063,0.000059,False,118,1,19
2,0.502652,0.638856,0.008003,0.822223,5.023191,0.639083,4.874802,8.586109,7.627805,0.423603,...,0.000083,0.000069,0.000035,0.000060,0.000074,0.000059,False,117,2,19
3,0.495819,0.627573,-0.025559,0.576451,5.174972,0.628535,4.286158,7.254839,6.819285,0.418430,...,0.000050,0.000046,0.000046,0.000037,0.000041,0.000059,False,116,3,19
4,0.500157,0.635425,0.005017,0.517341,5.137364,0.635419,3.987098,7.636392,6.274751,0.421225,...,0.000039,0.000099,0.000050,0.000070,0.000072,0.000059,False,115,4,19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114,2.598606,3.631924,-0.326276,2.178601,5.135965,3.632455,20.789969,40.855694,5.723393,2.069754,...,0.000044,0.000623,0.000376,0.000167,0.000164,0.000282,False,5,114,19
115,2.760711,3.819661,-0.216943,1.819169,5.209615,3.819631,20.881772,40.246332,5.466961,2.208666,...,0.000427,0.000266,0.000072,0.000223,0.000414,0.000297,False,4,115,19
116,2.900586,3.987385,-0.232640,1.754787,5.082078,3.987478,24.785411,48.041010,6.215812,2.334231,...,0.000384,0.000170,0.000230,0.000099,0.000457,0.000310,False,3,116,19
117,3.172205,4.339416,-0.278732,1.564076,5.309246,4.339480,21.908450,41.713333,5.048635,2.556862,...,0.000597,0.000421,0.000286,0.000399,0.000236,0.000340,False,2,117,19


In [89]:
res = series_to_moving_average(data, window_len=4, lag=3, lbl_cols=[])
res['B19_Survival_time'] = res['B19_Survival_time']
res

,B19_mean,B19_std,B19_skew,B19_kurtosis,B19_entropy,B19_rms,B19_max,B19_p2p,B19_crest,B19_clearence,...,B19_Fo,B19_Fi,B19_Fr,B19_Frp,B19_Fca,B19_noise,B19_Event,B19_Survival_time,Time,Id
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114,2.583945,3.565877,-0.229593,1.882974,5.108621,3.565845,21.382323,41.730091,5.997109,2.071847,...,0.000300,0.000238,0.000406,0.000314,0.000167,0.000281,0.0,9.5,109.5,19.0
115,2.610006,3.604292,-0.234163,1.893537,5.124394,3.604260,21.339321,41.515824,5.920295,2.091485,...,0.000342,0.000225,0.000370,0.000231,0.000223,0.000284,0.0,8.5,110.5,19.0
116,2.590291,3.591992,-0.262664,1.996971,5.113292,3.591952,22.014165,41.659829,6.130424,2.070807,...,0.000359,0.000292,0.000288,0.000192,0.000280,0.000282,0.0,7.5,111.5,19.0
117,2.588439,3.602730,-0.285075,2.092864,5.107670,3.602832,22.075468,41.863555,6.130155,2.065415,...,0.000201,0.000430,0.000265,0.000191,0.000298,0.000282,0.0,6.5,112.5,19.0
